# **Pryecto Data Analytics: Siniestros Viales**

### ETL: Primer Acercamiento a los Datos

Datos extraídos de base web de datos públicos, generados, guardados y publicados por el Gobierno de la Ciudad de Buenos Aires.

Sitio web:
[Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales)

Libros de Excel descargados y guardados dentro de [data](./data/) en el respositorio donde se encuentra este Notebook.

- El libro de homicidios está conformado por hojas que contienen los datos; hojas de "HECHOS" con la información de los siniestros y "VICTIMAS" con información sobre las victimas mortales; cada uno con sus respectivos diccionarios para las definiciones y descripciones de las variables.

- Se especifica qué valores representan datos faltantes: De acuerdo a las definiciones, "SD" quiere decir sin dato. 

In [60]:
import pandas as pd
import numpy as np 

# Leer y convertir a Datrafreame
df_hechos = pd.read_excel(
    './data/homicidios.xlsx',
    sheet_name='HECHOS',    # Tabla en la hoja HECHOS
    # na_values=['SD','Sd']
)
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [61]:
# Leer y convertir a DataFrame
df_victimas = pd.read_excel(
    './data/homicidios.xlsx',
    sheet_name='VICTIMAS',    # Tabla en la hoja VICTIMAS
    # na_values=['SD','Sd']
)
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


- Un total de **696** registros y **21** columnas para la tabla de hechos.

- Un total de **717** filas y **10** columnas para la tabla de víctimas.

Según las notas que acompañan los archivos, las tablas se relacionan entre sí mediante el ID del hecho y debe haber registros únicos de ID para cada siniestro en la tabla de Hechos.

In [62]:
# El largo de la lista de valores únicos para ID debería ser igual 
# al número de filas; 696
len(df_hechos['ID'].unique())

696

- La tabla de hechos presenta valores faltantes en las columnas de **Calle** y **Altura**. Por practicidad se determina que, de acuerdo con el objetivo del proyecto, no es necesario contar con parámetros tan específicos para las variables espaciales que describen en donde se produjo el siniestro. Es por esto que '**nombre de la calle**', '**altura**' y '**nombre del cruc**e' en que caso de que lo haya, no se considerarán relevantes y sus respectivas columnas serán descartadas para análisis posteriores.

- Aunque se quiere evitar perder información, para propositos de análisis es redundante tener dos columnas de dirección. Se eliminará la '**Dirección Normalizada**'.

- Se descartarán las columnas de '**Año**', '**Mes**' y '**Día**'. También la '**Hora**' y en su lugar se conservará '**HH**' que corresponde a la franja horaria.

- Se usará las coordenadas del sistema geodésico, por lo que '**XY (CABA)**' será eliminada.

- La columna '**PARTICIPANTES**' es una conjunción de las columnas de '**VICITMA**' y '**ACUSADO**'. Se descartará la primera mencionada.

In [63]:
columnas = ['Calle', 'Altura', 'Cruce', 'Dirección Normalizada',
            'AAAA', 'MM', 'DD', 'HORA', 'XY (CABA)', 'PARTICIPANTES']

df_hechos.drop(columns=columnas, inplace=True)

In [66]:
df_hechos.head()

,ID,N_VICTIMAS,FECHA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.46503904,-34.68092974,MOTO,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS


**Detección de valores nulos.**

Se considerará nulos los valores:
- Los Espacios en blanco ("").
- Se detectó que hay registros sin valores para Longitud y Latitud. En dichas filas aparece un punto (".") que tambíen será tomado como nulo.

Consideraciones:
- Para las variables **categóricas**/**cualitativas**: Si hay valores nulos y el tipo de dato más conveniente es un cadena de texto entonces los valores nulos o faltantes se dejarán de acuerdo a la notación original con que se cargaron los datos; como "**Sd**", queriendo decir Sin dato.

- Para la variables cuantitativas: En caso de existir un valor nulo para una variable numérica y cuyo tipo más adecuado sea un flotante entonces se reemplaza por `np.NaN`.

In [67]:
# Pos x, Pos y son numericas y deben ser float
df_hechos[['pos x', 'pos y']] = df_hechos[['pos x', 'pos y']].replace('.', np.nan).astype('float')

# Reemplazar espacios en blanco por SD
df_hechos.replace(
   { "": "SD"},
    inplace=True
)

**Normalizar las columnas que contienen textos.**

In [69]:
# Lista de columnas tipo texto
cols = ['LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

# Bucle para aplicar los cambios.
for col in cols:
    df_hechos[col] = (
        df_hechos[col]
        .map(str.strip, na_action='ignore')
        .map(str.title, na_action='ignore'))


**Renombrar las columnas**

In [70]:
old = df_hechos.columns
new = ['ID_hecho', 'Cantidad_Víctimas', 'Fecha', 'Hora', 'Lugar_del_hecho', 
       'Tipo_de_calle', 'Comuna', 'Longitud', 'Latitud', 'Víctima', 'Acusado']

renamer = {old: new for (old, new) in zip(old, new)}
df_hechos.rename(columns=renamer, inplace=True)

¿Cuáles **comunas** están presentes en el DataFrame?

In [79]:
Listado_comunas = df_hechos["Comuna"].unique().tolist()

print(f'Comunas donde hubo siniestros fatales: {Listado_comunas}')

Comunas donde hubo siniestros fatales: [8, 9, 1, 11, 15, 4, 7, 12, 3, 13, 14, 10, 6, 2, 5, 0]


No se supone que deba haber columna 0 ya que las comunas van de la 1 a la 15.

Se reemplazará el valor de cero por la notación de faltante "Sd"

In [80]:
df_hechos['Comuna'].replace(0,'Sd', inplace=True)

Version final del DataFrame.

In [81]:
df_hechos.head()

,ID_hecho,Cantidad_Víctimas,Fecha,Hora,Lugar_del_hecho,Tipo_de_calle,Comuna,Longitud,Latitud,Víctima,Acusado
0,2016-0001,1,2016-01-01,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.475340,-34.687570,Moto,Auto
1,2016-0002,1,2016-01-02,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.508775,-34.669777,Auto,Pasajeros
2,2016-0003,1,2016-01-03,7,Av Entre Rios 2034,Avenida,1,-58.390403,-34.631894,Moto,Auto
3,2016-0004,1,2016-01-10,0,Av Larrazabal Y Gral Villegas Conrado,Avenida,8,-58.465039,-34.680930,Moto,Sd
4,2016-0005,1,2016-01-21,5,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.387183,-34.622466,Moto,Pasajeros


Un tratamiento similar se le da a la tabla de víctimas. Se aplican los procedimientos bajo las mismas premisas.

In [82]:
# Columnas a descartar
columnas = ['AAAA', 'MM', 'DD']

# Eliminar
df_victimas.drop(columns=columnas, inplace=True)

# Aplicar normalización al texto
cols = ['ROL', 'VICTIMA', 'SEXO']

for col in cols:
    # Eliminar espacios en blanco innecesarios si los hay
    # Aplicar método title al texto.
    df_victimas[col] = df_victimas[col].apply(str.strip).apply(str.title)

# Renombrar las columnas
olds = df_victimas.columns
news = ['Fecha', 'Rol', 'Víctima', 'Sexo', 'Edad', 'Fecha_Fallecimiento']

# Diccionario para pasar por el parámetro y renombrar las columnas
renamer = {old: new for (old, new) in zip(olds[1:], news)}    # Se excluye la primera columna
df_victimas.rename(columns=renamer, inplace=True)

df_victimas.head()

,ID_hecho,Fecha,Rol,Víctima,Sexo,Edad,Fecha_Fallecimiento
0,2016-0001,2016-01-01,Conductor,Moto,Masculino,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,Conductor,Auto,Masculino,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,Conductor,Moto,Masculino,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,Conductor,Moto,Masculino,18,SD
4,2016-0005,2016-01-21,Conductor,Moto,Masculino,29,2016-02-01 00:00:00


In [85]:
# Llenar espacios en blanco con SD
df_victimas.replace(
    {
        "": "Sd",
    },
    inplace=True
)

Según el diccionario de datos, las columnas para **Acusado** y **Victimas** se refieren al vehiculo que ocupaba **quien haya sido acusado** y **quien haya fallecido** respectivamente. Es una buena idea agregarle un prefijo al nombre de aquellas columnas para hacerlo más explícito y entendible que se trata del tipo de vehículo involucrado.

In [86]:
# Cambiando el nombre en hechos
df_hechos.rename(
    columns={
        'Acusado': 'Vehiculo_Acusado',
        'Víctima': 'Vehículo_Víctima'
    },
    inplace=True
)

# Cambiando el nombre en victimas
df_victimas.rename(
    columns={
        'Víctima': 'Vehículo_Víctima'
    },
    inplace=True
)

# Cambiando columna de fechas de fallecimiento a formato datetime                                                                            
df_victimas['Fecha_Fallecimiento'] = pd.to_datetime(
    df_victimas['Fecha_Fallecimiento'],
    errors='coerce'
    )

Guardar en formato csv.

In [87]:
df_hechos.to_csv('./data/homicidios_hechos.csv', index=False)
df_victimas.to_csv('./data/homicidios_victimas.csv', index=False)